<a href="https://colab.research.google.com/github/ioannidisg/Machine-Learning/blob/main/problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [ ]:
NAME = "Ioannidis Georgios"
AEM = "10490"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Φόρτωση του συνόλου δεδομένων
data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)



In [ ]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.estimator = estimator
        self.models = []
        self.n_features = None

    def fit(self, X_train, y_train):
        self.n_features = X_train.shape[1]
        self.models = []

        for i in range(self.n_features):
            new_X_train = np.delete(X_train, i, axis=1) # εδώ αφαιρώ την i μεταβλητή εισόδου
            temp_mod = clone(self.estimator) # αντίγραφο βασικού μοντέλου
            temp_mod.fit(new_X_train, y_train) #
            self.models.append(temp_mod)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], self.n_features)) # πίνακας μηδενικών

        for i, model in enumerate(self.models): # loop για κάθε μοντέλο
            new_X = np.delete(X, i, axis=1) # για αντιστοίχηση με fit
            predictions[:, i] = model.predict(new_X)

        # Συνδυασμός των προβλέψεων με ψηφοφορία
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)
        #arr="πινακας που θα εφαρμοστεί η συνάρτηση" ,argmax="θεση μεγαλύτερης τιμής"
        return final_predictions


In [ ]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.n_estimators = n_estimators
        self.estimator = estimator
        self.models = []
        self.alphas = []

    def fit(self, X_train, y_train):Εργασία 3
Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:


[ ]
NAME = "Ioannidis Georgios"
AEM = "10490"
1 Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)


[ ]
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Φόρτωση του συνόλου δεδομένων
data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)



[ ]
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116
2 Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)


[ ]
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.estimator = estimator
        self.models = []
        self.n_features = None

    def fit(self, X_train, y_train):
        self.n_features = X_train.shape[1]
        self.models = []

        for i in range(self.n_features):
            new_X_train = np.delete(X_train, i, axis=1) # εδώ αφαιρώ την i μεταβλητή εισόδου
            temp_mod = clone(self.estimator) # αντίγραφο βασικού μοντέλου
            temp_mod.fit(new_X_train, y_train) #
            self.models.append(temp_mod)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], self.n_features)) # πίνακας μηδενικών

        for i, model in enumerate(self.models): # loop για κάθε μοντέλο
            new_X = np.delete(X, i, axis=1) # για αντιστοίχηση με fit
            predictions[:, i] = model.predict(new_X)

        # Συνδυασμός των προβλέψεων με ψηφοφορία
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)
        #arr="πινακας που θα εφαρμοστεί η συνάρτηση" ,argmax="θεση μεγαλύτερης τιμής"
        return final_predictions


[ ]
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006
3 Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)


[ ]


        n_samples = X_train.shape[0]
        sample_weight = np.ones(n_samples) / n_samples     # Αρχικοποίω τα βάρη

        for t in range(self.n_estimators):

            model = clone(self.estimator) # Δημιουργία αντιγράφου
            model.fit(X_train, y_train, sample_weight=sample_weight)

            y_pred = model.predict(X_train)
            incorrect = (y_pred != y_train).astype(int)  # Υπολογισμός των εσφαλμένων ταξινομημένων παραδειγμάτων

            weighted_error = np.dot(sample_weight, incorrect) / np.sum(sample_weight) #σταθμισμένο σφάλμα

            if weighted_error >= 0.5 or weighted_error == 0:
                break

            # Υπολογισμός του alpha
            alpha = 0.5 * np.log((1 - weighted_error) / (weighted_error + 1e-10))
            self.models.append(model)
            self.alphas.append(alpha)


            sample_weight *= np.exp(alpha * incorrect * 2 - alpha) # Ενημέρωση των βαρών
            sample_weight /= np.sum(sample_weight)  # Κανονικοποίηση των βαρών

    def predict(self, X):

        model_preds = np.array([model.predict(X) for model in self.models])  # Πρόβλεψη από όλα τα μοντέλα

        # Υπολογισμός σταθμισμένων ψήφων
        weighted_votes = np.zeros((X.shape[0], len(np.unique(y_train))))
        for alpha, preds in zip(self.alphas, model_preds):
            for i, pred in enumerate(preds):
                weighted_votes[i, pred] += alpha

        # Επιλογή της κλάσης με την υψηλότερη σταθμισμένη ψήφο
        return np.argmax(weighted_votes, axis=1)

In [ ]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
print(round (accuracy_score(ab.predict(X_test), y_test), 4))
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


0.9591


In [ ]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591